In [151]:
import numpy as np
import pandas as pd
import gensim
import importlib

from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from tensorflow.python.keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.python.keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from tensorflow.python.keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import Model
from tensorflow.python.keras import backend as K
from keras.engine.topology import Layer
from tensorflow.python.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.python.keras import initializers
from tensorflow.python.keras.engine import InputSpec, Layer
from tensorflow.python.keras import backend as K

import spacy
from spelling_check import spell_check
from pretrain import load_pretrain 
from tqdm import tqdm_notebook as tqdm
import gc

import clean_data
import create_table
from sklearn.model_selection import train_test_split
import keras

import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [3]:
%load_ext autoreload
%autoreload 2

In [8]:
train = pd.read_csv('train.csv').fillna(' ')
train_text = train['question_text'].tolist()
y = train['target'].values
num_train_data = y.shape[0]

In [5]:
Table = create_table.uncommon_char_table()

In [12]:
text_list = []
for d in tqdm(train_text):
    text_list.append(clean_data.normalize(d))

100%|██████████| 1306122/1306122 [03:35<00:00, 6061.20it/s]


In [47]:
for i in tqdm(range(len(text_list))):
    text_list[i] = text_list[i].split()
    for j in range(len(text_list[i])):
        try:
            if text_list[i][j][0].isupper() and text_list[i][j][1].islower():
                text_list[i][j] = text_list[i][j].lower()
        except:
            continue 
    text_list[i] = " ".join(text_list[i])

100%|██████████| 1306122/1306122 [00:06<00:00, 217395.54it/s]


In [49]:
nlp = spacy.load('en_core_web_lg')
nlp.vocab.add_flag(lambda s: s.lower() in spacy.lang.en.stop_words.STOP_WORDS, spacy.attrs.IS_STOP)
word_dict = {}
word_index = 1
lemma_dict = {}
docs = nlp.pipe(text_list, n_threads = 2)
word_sequences = []

In [50]:
for doc in tqdm(docs):
    word_seq = []
    for token in doc:
        if (token.text not in word_dict) and (token.pos_ is not "PUNCT"):
            word_dict[token.text] = word_index
            word_index += 1
            lemma_dict[token.text] = token.lemma_
        if token.pos_ is not "PUNCT":
            word_seq.append(word_dict[token.text])
    word_sequences.append(word_seq)
del docs
gc.collect()

1306122it [38:16, 568.80it/s]


0

In [52]:
word_sequences = pad_sequences(word_sequences, maxlen=55, padding='post')

In [56]:
X_train, X_test, y_train, y_test = train_test_split(word_sequences, y, test_size=0.25, random_state=2019)

In [64]:
y_train = keras.utils.to_categorical(y_train,2)

In [59]:
spell_model = gensim.models.KeyedVectors.load_word2vec_format('embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec')
words = spell_model.index2word
w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i
WORDS = w_rank

In [60]:
embedding_matrix_glove= load_pretrain('embeddings/glove.840B.300d/glove.840B.300d.txt',word_dict, lemma_dict,'Glove',WORDS)

100%|██████████| 233830/233830 [00:43<00:00, 5426.31it/s]


In [61]:
embedding_matrix_fasttext = load_pretrain('embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec',word_dict, lemma_dict,'word2vec',WORDS)

100%|██████████| 233830/233830 [00:26<00:00, 8847.62it/s] 


In [65]:
embedding_matrix = np.concatenate((embedding_matrix_glove, embedding_matrix_fasttext), axis=1)

In [71]:
inp = Input(shape=(55,))
x = Embedding(len(word_dict)+1, 600, weights=[embedding_matrix], trainable=False)(inp)
x = SpatialDropout1D(0.3)(x)
x1 = Bidirectional(LSTM(128, return_sequences=True))(x)
x2 = Bidirectional(GRU(128, return_sequences=True))(x1)
max_pool1 = GlobalMaxPooling1D()(x1)
max_pool2 = GlobalMaxPooling1D()(x2)
conc = Concatenate()([max_pool1, max_pool2])
predictions = Dense(2, activation='softmax')(conc)
model = Model(inp, predictions)
adam = optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [72]:
model.fit(X_train, y_train, batch_size=512, epochs=3, verbose=1)

W1003 16:43:25.653223 4693046720 deprecation.py:323] From /Users/sizhenhan/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 979591 samples
Epoch 1/3
979591/979591 [==============================] - 8838s 9ms/sample - loss: 0.1143 - accuracy: 0.9553
Epoch 2/3
979591/979591 [==============================] - 5721s 6ms/sample - loss: 0.1008 - accuracy: 0.9598
Epoch 3/3
979591/979591 [==============================] - 5782s 6ms/sample - loss: 0.0947 - accuracy: 0.9622


In [80]:
pred = model.predict(X_test)

In [177]:
res = []
for p in pred:
    if p[0] > 0.6:
        res.append(0)
    else:
        res.append(1)

In [178]:
accuracy_score(res,y_test)

0.9599946100064006

In [179]:
precision_score(res,y_test,pos_label=1)

0.701176586422488

In [129]:
y_train2 = []
for y in y_train:
    if y[0] == 1:
        y_train2.append(0)
    else:
        y_train2.append(1)

In [138]:
y_train2 = np.array(y_train2)
y_train2 = y_train2.reshape((len(y_train2),1))

In [140]:
x_train_torch = torch.tensor(X_train, dtype=torch.long)
x_test_torch = torch.tensor(X_test, dtype=torch.long)
y_train_torch = torch.tensor(y_train2, dtype=torch.float32)
features = len(word_dict)+1

In [141]:
train_dataset = data.TensorDataset(x_train_torch, y_train_torch)
test_dataset = data.TensorDataset(x_test_torch)

In [142]:
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    
        x = x.permute(0, 3, 2, 1) 
        x = super(SpatialDropout, self).forward(x)  
        x = x.permute(0, 3, 2, 1) 
        x = x.squeeze(2) 
        return x

    

In [147]:
class NeuralNet(nn.Module):
    def __init__(self, embedding_matrix):
        super(NeuralNet, self).__init__()
        self.embedding = nn.Embedding(features, 600)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = SpatialDropout(0.3)
        
        self.lstm1 = nn.LSTM(600, 128, bidirectional=True, batch_first=True)
        self.gru = nn.GRU(128 * 2, 128, bidirectional=True, batch_first=True)
        self.linear_out = nn.Linear(512, 1)
        
    def forward(self, x):
        h_embedding = self.embedding(x)
        h_embedding = self.embedding_dropout(h_embedding)
        
        h_lstm1, _ = self.lstm1(h_embedding)
        h_gru, _ = self.gru(h_lstm1)
        
        max_pool1,_ = torch.max(h_lstm1, 1)
        max_pool2, _ = torch.max(h_gru, 1)
        
        h_conc = torch.cat((max_pool1, max_pool2), 1)
        out  = self.linear_out(h_conc)
        return out 

In [148]:
model = NeuralNet(embedding_matrix)

In [152]:
def train_model(model, train, loss_fn, lr=0.001,
                batch_size=512, n_epochs=3):
    param_lrs = [{'params': param, 'lr': lr} for param in model.parameters()]
    optimizer = torch.optim.Adam(param_lrs, lr=lr)
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    
    for epoch in range(n_epochs):
        model.train()
        avg_loss = 0.
        
        for x_batch, y_batch in tqdm(train_loader, disable=False):
            y_pred = model(x_batch)            
            loss = loss_fn(y_pred, y_batch)

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            avg_loss += loss.item() / len(train_loader)
        
        print('Epoch {}/{} \t loss={:.4f}'.format(
              epoch + 1, n_epochs, avg_loss))
        
    return model 


In [153]:
train_model(model, train_dataset, loss_fn=nn.BCEWithLogitsLoss(reduction='mean'))

Epoch 1/3 	 loss=0.1105


Epoch 2/3 	 loss=0.0997


Epoch 3/3 	 loss=0.0941


NeuralNet(
  (embedding): Embedding(233831, 600)
  (embedding_dropout): SpatialDropout(p=0.3, inplace=False)
  (lstm1): LSTM(600, 128, batch_first=True, bidirectional=True)
  (gru): GRU(256, 128, batch_first=True, bidirectional=True)
  (linear_out): Linear(in_features=512, out_features=1, bias=True)
)

In [158]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=512, shuffle=False)
test_preds = np.zeros((len(X_test), 1))

In [115]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [161]:
for i, x_batch in enumerate(test_loader):
    y_pred = sigmoid(model(*x_batch).detach().cpu().numpy())

    test_preds[i * 512:(i+1) * 512, :] = y_pred
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [163]:
test_preds[:20]

array([[3.52622592e-04],
       [2.17369781e-03],
       [1.45465764e-03],
       [3.92834227e-05],
       [4.64717334e-04],
       [2.00470924e-04],
       [5.79589338e-04],
       [5.28251752e-04],
       [2.73891841e-03],
       [1.24002923e-03],
       [5.74536741e-01],
       [3.09422798e-03],
       [2.81704846e-03],
       [2.18538975e-04],
       [3.27847041e-02],
       [2.72917515e-03],
       [4.40425792e-05],
       [1.34231837e-03],
       [2.93648848e-03],
       [2.34954478e-03]])

In [171]:
res2 = []
for p in test_preds:
    if p[0] > 0.4:
        res2.append(1)
    else:
        res2.append(0)

In [172]:
accuracy_score(res2,y_test)

0.9603467970881785

In [173]:
precision_score(res2,y_test,pos_label=1)

0.6452518091862354

In [181]:
inp = Input(shape=(55,))
x = Embedding(len(word_dict)+1, 600, embeddings_initializer = 'uniform')(inp)
x = SpatialDropout1D(0.3)(x)
x1 = Bidirectional(LSTM(128, return_sequences=True))(x)
x2 = Bidirectional(GRU(128, return_sequences=True))(x1)
max_pool1 = GlobalMaxPooling1D()(x1)
max_pool2 = GlobalMaxPooling1D()(x2)
conc = Concatenate()([max_pool1, max_pool2])
predictions = Dense(2, activation='softmax')(conc)
model = Model(inp, predictions)
adam = optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [182]:
model.fit(X_train, y_train, batch_size=512, epochs=3, verbose=1)

/Users/sizhenhan/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:414: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 979591 samples
Epoch 1/3
979591/979591 [==============================] - 49087s 50ms/sample - loss: 0.1189 - accuracy: 0.9536
Epoch 2/3
979591/979591 [==============================] - 12226s 12ms/sample - loss: 0.0937 - accuracy: 0.9623
Epoch 3/3
979591/979591 [==============================] - 12243s 12ms/sample - loss: 0.0747 - accuracy: 0.9705


In [183]:
pred2 = model.predict(X_test)

In [184]:
res = []
for p in pred2:
    if p[0] > 0.6:
        res.append(0)
    else:
        res.append(1)

In [185]:
accuracy_score(res,y_test)

0.95447599155976

In [186]:
precision_score(res,y_test,pos_label=1)

0.5631861369566288